In [1]:
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp

In [2]:
from mnist import MNIST
print("Loading dataset....")
mndata = MNIST('../mnist_dataset')
X_train, y_train = mndata.load_training()
X_train = (mndata.process_images_to_numpy(X_train)/255).astype(np.float32,copy=False)
y_train = np.asarray(y_train,dtype=np.uint8)
X_test, y_test = mndata.load_testing()
X_test = (mndata.process_images_to_numpy(X_test)/255).astype(np.float32,copy=False)
y_test = np.asarray(y_test,dtype=np.uint8)
print("Done.")

Loading dataset....
Done.


In [3]:
def one_hot_encode(y):
    Yo=np.zeros((y.size,int(y.max())+1),dtype=np.float32)
    Yo[np.arange(y.size),y]=1
    return Yo

In [4]:
y_train=one_hot_encode(y_train)
y_test=one_hot_encode(y_test)

In [5]:
from nnet_gpu.network import Sequential,layers
from nnet_gpu.layers import dense
from nnet_gpu import functions
from nnet_gpu import optimizers
import numpy as np

Seed: 305


In [6]:
model=Sequential()
model.add(dense(2048,input_shape=(784),activation=functions.relu))
model.add(dense(1024,activation=functions.relu))
model.add(dense(512,activation=functions.relu))
model.add(dense(256,activation=functions.relu))
model.add(dense(128,activation=functions.relu))
model.add(dense(10,activation=functions.softmax))

In [7]:
model.summary()

⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽⎽
Layer (type)               Output Shape             Activation        Param #
- input_layer(InputLayer) (None, 784)                echo             0
__________________________________________________________________________________________
0 dense(dense)            (None, 2048)               relu             1607680
__________________________________________________________________________________________
1 dense(dense)            (None, 1024)               relu             2098176
__________________________________________________________________________________________
2 dense(dense)            (None, 512)                relu             524800
__________________________________________________________________________________________
3 dense(dense)            (None, 256)                relu             131328
_____________________________________________________________________________________

In [8]:
model.compile(optimizer=optimizers.iterative,loss=functions.cross_entropy_with_logits,learning_rate=0.001)

In [9]:
X_train=cp.asarray(X_train)
y_train=cp.asarray(y_train)
X_test=cp.asarray(X_test)
y_test=cp.asarray(y_test)

In [10]:
model.fit(X_train,y_train,batch_size=256,epochs=10,validation_data=(X_test,y_test))

EPOCH: 1 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2095 - accuracy: 0.1171      __
Epoch time: 6.429s
Validation Accuracy: 0.113 - val_loss: 4.7832
EPOCH: 2 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2160 - accuracy: 0.0988      _
Epoch time: 6.209s
Validation Accuracy: 0.113 - val_loss: 4.7833
EPOCH: 3 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2080 - accuracy: 0.1137      _
Epoch time: 6.228s
Validation Accuracy: 0.113 - val_loss: 4.7830
EPOCH: 4 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2032 - accuracy: 0.1124      _
Epoch time: 6.259s
Validation Accuracy: 0.113 - val_loss: 4.7839
EPOCH: 5 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2141 - accuracy: 0.1095      _
Epoch time: 6.225s
Validation Accuracy: 0.113 - val_loss: 4.7832
EPOCH: 6 / 10
Progress:  60160 / 60000  - 0s - 0.024s/sample - loss: 2.2042 - accuracy: 0.1214      _
Epoch time: 6.202s
Validation Accuracy: 0.113 - val_loss: 4.78

In [10]:
model.fit(X_train,y_train,batch_size=256,epochs=10,validation_data=(X_test,y_test))

EPOCH: 1 / 10
Progress:  60160 / 60000  - 0s - 0.0077s/sample - loss: 0.1658 - accuracy: 0.9531      _
Epoch time: 14.927s
Validation Accuracy: 0.950 - val_loss: 0.3488
EPOCH: 2 / 10
Progress:  60160 / 60000  - 0s - 0.0081s/sample - loss: 0.1063 - accuracy: 0.9675      _
Epoch time: 14.723s
Validation Accuracy: 0.962 - val_loss: 0.2618
EPOCH: 3 / 10
Progress:  60160 / 60000  - 0s - 0.0076s/sample - loss: 0.0987 - accuracy: 0.9783      _
Epoch time: 14.833s
Validation Accuracy: 0.972 - val_loss: 0.2042
EPOCH: 4 / 10
Progress:  25344 / 60000  - 1s - 0.0078s/sample - loss: 0.1693 - accuracy: 0.9799      _

KeyboardInterrupt: 